In [1]:
import numpy as np
import pandas as pd
from datetime import date, datetime
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
fileList = ['listings.csv', 'listings_20240805.csv', 'listings_20240705.csv','listings_20240604.csv',
            'listings_20240503.csv','listings_20240406.csv','listings_20240307.csv','listings_20240206.csv',
            'listings_20240105.csv','listings_20231204.csv','listings_20231101.csv','listings_20231001.csv']

df = pd.concat((pd.read_csv(i) for i in fileList)).reset_index(drop = True)

In [3]:
df.drop(columns=['listing_url', 'scrape_id', 'source', 'name',
                 'description', 'neighborhood_overview', 'picture_url', 'host_id',
                 'host_url', 'host_name', 'host_location', 'host_about',
                 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
                 'host_listings_count', 'host_total_listings_count', 'host_verifications',
                 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
                 'calendar_updated','first_review','last_review','calendar_last_scraped'], inplace=True)

Data Cleaning and Processing

In [ ]:
df['host_response_rate'] = df['host_response_rate'].str.replace('%','').astype(float).apply(lambda x: x/100)
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%','').astype(float).apply(lambda x: x/100)

In [4]:
df['price'] = df['price'].str.replace(r'\$|,', '', regex=True).astype(float)

In [5]:
df.loc[df['price'] == 0 ,'price'] = np.nan
df['price'] = df.groupby(['neighbourhood_group_cleansed','room_type'])['price'].transform(lambda x: x.fillna(x.median()))

In [6]:
df['bedrooms'] = df.groupby(['room_type'])['beds'].transform(lambda x: x.fillna(x.median()))

In [7]:
# get the bathroom values from the bathroom_text feature
df['bathrooms_text'] = df['bathrooms_text'].str.replace(r'\s.*', '', regex=True).apply(lambda x: 0.5 if x in ('Half-bath', 'Shared','Private') else float(x)).astype(float)
# fill the na in the bathroom feature with the value from the bathroom_text feature
df['bathrooms_cleaned'] = df['bathrooms'].fillna(df['bathrooms_text'])
df.drop(columns=['bathrooms_text','bathrooms'],inplace=True)

In [9]:
values = {'host_is_superhost': 'f', 'host_has_profile_pic' :'f', 'host_identity_verified':'f', 'has_availability':'f','instant_bookable':'f',
          'reviews_per_month': df['reviews_per_month'].median(), 'number_of_reviews':df['number_of_reviews'].median(), 
          'number_of_reviews_ltm':df['number_of_reviews_ltm'].median(), 'number_of_reviews_l30d':df['number_of_reviews_l30d'].median(), 
          'bedrooms':1, 'beds': df['beds'].median(), 'bathrooms_cleaned': df['bathrooms_cleaned'].median(),
          'review_scores_rating':df['review_scores_rating'].median(), 'review_scores_accuracy': df['review_scores_accuracy'].median(),
          'review_scores_cleanliness': df['review_scores_cleanliness'].median(), 'review_scores_checkin': df['review_scores_checkin'].median(),
          'review_scores_communication': df['review_scores_communication'].median(), 'review_scores_location': df['review_scores_location'], 
          'review_scores_value': df['review_scores_value'].median(), 'host_response_rate': df['host_response_rate'].median(), 
          'host_acceptance_rate' :df['host_acceptance_rate'].median(),'host_response_time': df['host_response_time'].value_counts().index[0]}
df.fillna(value=values, inplace=True)

df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0}).astype(int)
df['has_availability'] = df['has_availability'].map({'t': 1, 'f': 0}).astype(int)
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0}).astype(int)
df['has_license'] = df['license'].fillna(-1).apply(lambda x: 0 if x == -1 else 1)

In [ ]:
df = df.astype(
    {
        'neighbourhood_group_cleansed': 'category',
        'room_type' : 'category',
        'host_response_time' : 'category',
        'neighbourhood_cleansed': 'category',
        'property_type': 'category'
    }
)
df['last_scraped'] = pd.to_datetime(df['last_scraped'])

Feature Engineering

In [15]:
# find length of time for host
df['host_since'] = pd.to_datetime(df['host_since'])
df['today'] = pd.to_datetime(date.today())
df['host_length_years'] = (df['today'] - df['host_since']) / np.timedelta64(1, 'D')
df['host_length_years'] = round((df['host_length_years'] *  0.0027379),2).astype(float)

df.fillna(value={'host_length_years': df['host_length_years'].median()}, inplace=True)

df.drop(columns=['today','host_since'], inplace = True)

In [17]:
df['amenities'] = df['amenities'].str.replace('"', '')
df['amenities'] = df['amenities'].str.replace('[', '')
df['amenities'] = df['amenities'].str.replace(']', '')
df['amenities'] = df['amenities'].str.replace(r'\\u.*','', regex = True)
df['amenities'] = df['amenities'].str.strip()
df['parking_on_premises'] = df['amenities'].str.contains(r'.*parking on premises|Free driveway parking.*|PAID.*PARKING.*ON PREMISES|.*PARKING GARAGE ON PREMISES|Free residential garage on premises|Free carport on premises', 
                                                                         case=False, regex=True).astype(int)
df['washer'] = df['amenities'].str.contains(r'WASHER|FREE WASHER|PAID WASHER', case=False, regex=True).astype(int)
df['dryer'] = df['amenities'].str.contains(r'DRYER|FREE DRYER|PAID DRYER', case=False, regex=True).astype(int)
df['AC'] = df['amenities'].str.contains(r'Central air conditioning|AIR CONDITIONING|Window AC unit|AC - split type ductless system', case=False, regex=True).astype(int)
df['heating'] = df['amenities'].str.contains(r'HEATING|CENTRAL HEATING|Radiant heating|Heating - split type ductless system', case=False, regex=True).astype(int)
df['wifi'] = df['amenities'].str.contains(r'WIFI|.*WIFI', case=False, regex=True).astype(int)
df['TV'] = df['amenities'].str.contains(r'TV|TV\s.*|.*HDTV.*', case=False, regex=True).astype(int)
df['self_check_in'] = df['amenities'].str.contains('Self check-in').astype(int)
df['gym'] = df['amenities'].str.contains(r'GYM|Shared gym in building|PRIVATE GYM IN BUILDING|Gym in building|Private gym|Shared gym', case=False, regex=True).astype(int)
df['pets_allowed'] = df['amenities'].str.contains('Pets allowed').astype(int)
df['kitchen'] = df['amenities'].str.contains(r'KITCHEN|KITCHENETTE', case=False, regex=True).astype(int)
df['patio_balcony'] = df['amenities'].str.contains(r'.*PATIO OR BALCONY|Patio or balcony', case=False, regex=True).astype(int)
df['backyard'] = df['amenities'].str.contains(r'.*BACKYARD.*', case=False, regex=True).astype(int)
df['pool'] = df['amenities'].str.contains(r'Pool|Shared pool.*|Private pool|Shared indoor pool - available all year|Shared outdoor pool - available seasonally', case=False, regex=True).astype(int)
df['luggage_dropoff_allowed'] = df['amenities'].str.contains('Luggage dropoff allowed').astype(int)
df['dedicated_workspace'] = df['amenities'].str.contains('Dedicated workspace').astype(int)
df['lock_on_bedroom_door'] = df['amenities'].str.contains('Lock on bedroom door').astype(int)

In [ ]:
df = pd.get_dummies(df,columns=['room_type','neighbourhood_group_cleansed','host_response_time'],dtype=int)

Correlation Analysis

In [20]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

corrs = df.corr(numeric_only=True)
corrs.drop(columns=['latitude','longitude'],inplace=True)

targetCor = corrs.drop('price_log')['price_log']

targetCor.loc[targetCor.abs().sort_values(ascending= False).index]

price                                           0.558868
accommodates                                    0.546141
beds                                            0.425464
bedrooms                                        0.425166
neighbourhood_group_cleansed_Manhattan          0.349101
longitude                                      -0.282884
minimum_nights_sqrt                            -0.272586
bathrooms_cleaned                               0.252338
minimum_minimum_nights_sqrt                    -0.246253
kitchen                                        -0.245276
gym                                             0.222957
self_check_in                                   0.212926
calculated_host_listings_count_private_rooms   -0.197845
neighbourhood_group_cleansed_Queens            -0.190435
instant_bookable                                0.186727
availability_90                                 0.182397
availability_365                                0.179805
calculated_host_listings_count_

In [ ]:
df.drop(columns=[
    'maximum_nights','calculated_host_listings_count','minimum_nights_avg_ntm',
    'number_of_reviews','host_response_rate','minimum_maximum_nights',
    'host_is_superhost','maximum_maximum_nights','review_scores_value',
    'maximum_nights_avg_ntm','review_scores_checkin','number_of_reviews_ltm','review_scores_communication',
    'host_length_years','review_scores_accuracy','reviews_per_month',
    'number_of_reviews_l30d','maximum_minimum_nights','review_scores_rating','minimum_minimum_nights',
    'has_availability','host_acceptance_rate','availability_30',
    'availability_60','review_scores_cleanliness','availability_90',
    'calculated_host_listings_count_shared_rooms', 'availability_365',
    'beds','license','neighbourhood_cleansed','amenities','minimum_nights_sqrt',
    'backyard','AC','dedicated_workspace','luggage_dropoff_allowed','parking_on_premises',
    'patio_balcony','TV','pool','washer','heating','dryer',
    'host_response_time_within a day','host_response_time_a few days or more','host_response_time_within an hour',
    'host_response_time_within a few hours', 'review_scores_location'
],inplace=True)

In [ ]:
df.to_csv('Airbnb_listings_cleaned.csv',header=True)